# Sankey diagram - Nomenclature hierarchy conflicts 

This jupyter notebook was used to generate sankey diagram representing assigned nomenclature at ranks up to kingdom for full-length sequences with a keyword `Streptomyces` in the FASTA header. 

Step 1: Set Up

In [ ]:
import pandas as pd
import seaborn as sns
import plotly
import plotly.graph_objs as go
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import colors as mcolors

Step 2: Load and prepare data. 

Here, we need to generate two dataframes:

    - Links dataframe with the following columns: source, target, count, and colour. 
    - Node dataframe with the following columns: ID, label and color. 
    
    
To get number of 'counts' for each link, we use `groupby` funcion from `pandas`.

In [ ]:
nomenclature_hierarchy = pd.read_csv("nomenclature_hierarchy_info.csv")

In [ ]:
links_data = nomenclature_hierarchy.groupby(["domain", "phylum"]).size().reset_index(name='count').rename(columns={"domain": "source", "phylum": "target"})
links_data_2 = nomenclature_hierarchy.groupby(["phylum", "class"]).size().reset_index(name='count').rename(columns={"phylum": "source", "class": "target"})
links_data_3 = nomenclature_hierarchy.groupby(["class", "order"]).size().reset_index(name='count').rename(columns={"class": "source", "order": "target"})
links_data_4 = nomenclature_hierarchy.groupby(["order", "family"]).size().reset_index(name='count').rename(columns={"order": "source", "family": "target"})
links_data_5 = nomenclature_hierarchy.groupby(["family", "genus"]).size().reset_index(name='count').rename(columns={"family": "source", "genus": "target"})

links_data = links_data.append(links_data_2)
links_data = links_data.append(links_data_3)
links_data = links_data.append(links_data_4)
links_data = links_data.append(links_data_5)

Next we can assign colour to each link

In [ ]:

def get_colours(items, sns_palette):
    """Assign colour for each item in a give list.
    Return dictionary with hexa colours keyed by the item in the provided list.
    
    :param items: list of item for which individual colours will be assigned
    :param sns_palette: name of a seaborn palette from which the colours will be picked
    """
    palette = cm.get_cmap(sns_palette, len(items))
    

    colors = []
    
    for _ in palette(range(len(items))):
        _[-1] = 0.8
        rgba_colors = mcolors.to_rgba(tuple(_))
        colors.append('rgba'+str(rgba_colors))
        
    
    return dict(zip(items, colors))

In [ ]:
colours = {}
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['domain']])), 'Blues'))
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['phylum']])), 'Purples'))
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['class']])), 'Reds'))
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['order']])), 'Oranges'))
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['family']])), 'YlGn'))
colours.update(get_colours(list(set([_ for _ in nomenclature_hierarchy['genus']])), 'Greens'))

In [ ]:
links_data['Link_Colour'] = links_data['source'].map(colours)

To force all node labels to appear on the same side (right of each node), we need to add additional layer that has no label, and all target nodes lead to it. 

In [ ]:
links_data_6 = pd.DataFrame([_ for _ in links_data_5['target']], columns =['source'])
links_data_6['target'] = 119
links_data_6['Link_Colour']= 'white'
links_data_6['count'] = 10**-10
links_data = links_data.append(links_data_6)

Next, we can start creating our node data.

First, we need to create dataframe which will have all unique items found in `nomenclature_hierarchy` dataframe, and assign an ID to each item.

In [ ]:
nomenclature = []
for col in nomenclature_hierarchy:
    if col != 'accession':
        for item in nomenclature_hierarchy[col]:
            nomenclature.append(item)
            
nomenclature_ID = {_[1]:_[0]for _ in enumerate(list(set(nomenclature)))}
node_data = pd.DataFrame(nomenclature_ID.items(), columns=['Label', 'ID'])

Next, we can add colour info.

In [ ]:
node_data['Colour'] = node_data['Label'].map(colours)

Again, we need to add information for our additional layer, and set it up to 'white'. 

In [ ]:
node_data.loc[len(node_data)] = ['',119,'white']

Now, we can convert our `source` and `taregt` data in `links_data` to our assigned IDs assigned in `node_data`. 

In [ ]:
node_ids = dict(zip(node_data['Label'], node_data['ID']))

In [ ]:
node_ids[119] = 119

In [ ]:
links_data['source'] = links_data['source'].map(node_ids)
links_data['target'] = links_data['target'].map(node_ids)

Lastly, we can modify our `Label` column in `node_data` to show number of counts.

In [ ]:
new_labels = {k:'<b>'+k+'</b>' +' ('+str(v)+')' for k, v in Counter(nomenclature).items()}

In [ ]:
node_data['Label'] = node_data['Label'].map(new_labels)

In [ ]:

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 1000,
      thickness = 2,
      line = dict(color = "black", width = 0.01),
      label = node_data['Label'],
      color = node_data['Colour']
    ),
    link = dict(
      source = links_data['source'].dropna(axis=0, how='any'), # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = links_data['target'].dropna(axis=0, how='any'),
      value = links_data['count'].dropna(axis=0, how='any'),
      color = links_data['Link_Colour'].dropna(axis=0, how='any'),
        
  ))])


for x_coordinate, column_name in enumerate(["Kingdom","Phylum","Class", 'Order', 'Family', 'Genus', '']):
    fig.add_annotation(
          x=x_coordinate,
          y=1.05,
          xref="x",
          yref="paper",
          text=column_name,
          showarrow=False,
          font=dict(
              size=16,
              color="Black"
              ),
          align="center",
          )


fig.update_layout(title_text="", plot_bgcolor='rgba(0,0,0,0)',font=dict(size = 10, color = 'black'), yaxis={'side': 'right', 'showgrid':False,'zeroline': False,'visible': False}, xaxis={'showgrid': False, 'zeroline': False,'visible': False})
fig.show()
fig.write_image("Fig_3.pdf", width=1980, height=1080)